In [1]:
import numpy as np
import pandas as pd 

numpay y pandas son librerias  fccc

# Plantilla

Es necesario ajustar las definiciones, las fuentes de los datos y posiblemente definiciones si la ENEMDU tiene una dimensión geográfica y temporal al mismo tiempo

In [ ]:
data = pd.read_csv('datos/archivo.csv') # para datos csv
data = pd.read_stata('datos/archivo.dta') # para bases de stata

## Revisar los datos

In [ ]:
data.describe

In [ ]:
data.types

In [ ]:
data.info()

## Corregimos los valores de ser necesario

In [ ]:
# Llenar valores perdidos con un dato (0)
data['columna'] = data['columna'].fillna(0)

## Calculamos ingreso bruto por persona

In [ ]:
data['ingr_b'] = data['ingreso trabajo principal'] + data['ingreso trabajo secundario']
data['ingr_b'].mean()

## Deflactamos el ingreso

In [ ]:
# Cargamos los datos del ipc mensual de Ecuador para el año usado
ipc = np.array([])
# Elegimos el año base
ipcb = 100

In [ ]:
# Asigna el IPC a cada mes
data['ipc'] = data['mes'].map(lambda x: ipc[x-1])

# Calculamos el deflactor
data['def'] = data['ipc']/ipcb
data['ingr_r'] = data['ingr_b'] * data['def']

## Calculo ingreso de los hogares

In [ ]:
data['ingr_h'] = data.groupby('identificador hogares')['ingr_r'].transform('sum')

## Sacamos edades negativas, mayores a 100 años y menores de 10

In [ ]:
data = data.loc[(data['edad'] > 10) & (data['edad'] < 100)]

## Ingreso descontando cargas familiares

Utilizando la metodología del autor dividimos el ingreso del hogar para la escala $(A_{i}+kC_{i})^{s}$ donde $A_{i}$ es al número de adultos, $C_{i}$ es el número de niños en el hogar $i$. $k$ es el costo en recursos de cada niño y $s$ busca reflejar las restricciones

In [ ]:
k = 0.4
s = 0.9

In [ ]:
# Si es necesario calcular el número de niños
data['es_nino'] = data['edad'] < 10

data['ninos'] = data.groupby('identificador hogares')['es_nino'].transform('sum')

# Si es necesario calcular el número de adultos
data['es_adulto'] = data['edad'] > 10

data['adultos'] = data.groupby('identificador hogares')['es_adulto'].transform('sum')

In [ ]:
data['escala'] = (data['adultos'] + k * data['ninos']) ** s

In [ ]:
data['ingr_t'] = data['ingr_h'] / data['escala']

## Umbrales de pobreza

Incluimos los índices de pobreza si es posible a nivel regional para luego poder utilizar de mejor forma el factor de expansión

In [ ]:
pobr = np.array()

# Asigna el umbral a cada región
data['umbral'] = data['región'].map(lambda x: pobr[x])

## Cálculo del índice de pobreza de Foster, Greer y Thorbecke

Para calcular un ínidce de pobreza se utiliza a Foster, Greer y Thorbecke (1984), ya que satisface algunas caracterísitcas de distribución que son positivas e igual a las enunciadas por Sen, el autor usa el mismo índice.

$FGT_{\alpha} = \frac{1}{N}\sum_{i=1}^{H}\left(\frac{z-y_{i}}{z}\right)^{\alpha} $

Donde $z$ es el umbral de pobreza, $N$ es el número de personas en la economía, $H$ es el número de pobres (personas debajo de la línea de pobreza) $y_{i}$ es el ingreso de cada individuo. Mientras mayor es el valor de $\alpha$ mayor es el peso de los individuos más pobres, mayor $FGT$ mayor pobreza en la economía.

In [ ]:
# Buscamos a los que están por debajo del umbral de pobreza
data['pobres'] = (data['ingr_t'] - data['umbral']) < 0

# Cantidad de pobres por región
data['pobres_region'] = data.groupby('identificador_region')['pobres'].transform('sum')

# Cantidad de personas por región
data['personas'] = data.groupby('identificador_region')['identificador persona'].transform('sum')

In [8]:
# Cálculo del FGT para alpha 0,1 y 2 dependiendo de las dimensiones de la ENEMDU...

## Calculo del ínidce de desigualdad de Atkinson

Vamos a calcular el índice de desigualdad de atkinson con un parámetro $\epsilon$ de aversión a la desigualdad y un $\mu$ que es igual a la media de los ingresos individuales, con la siguiente fórmula.

$A = 1-\frac{1}{\mu}\left(\frac{1}{N}\sum_{i=1}^{N}y^{1-\epsilon}\right)^{1/(1-\epsilon}$

Donde $y_{i}$ es el ingreso individual y $\mu$ es el ingreso medio

In [ ]:
# Ingreso promedio por región
data['mu'] = data.groupby('identificador_region')['ingr_t'].transform('mean')

In [9]:
# Cálculo del A para epsilon 0.25, 0.5 y 0.75 dependiendo de las dimensiones de la ENEMDU...